# Install compatible pip version

In [ ]:
pip install --upgrade pip==24.0

Eri�im engellendi.


In [1]:
pip install trdg

  Using cached trdg-1.8.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.7 kB ? eta -:--:--
     --------------------- ------------------ 30.7/57.7 kB 1.0 MB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.7 kB 494.3 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 530.0 kB/s eta 0:00:00
  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached diffimg-0.2.3.tar.gz (4.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached arabic_reshaper-2.1.3-py3-none-any.whl.metadata (12 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     -------------------------- ----

Reason for being yanked: Doesn't work with Python 2

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
def fix_utils_file():
    import trdg
    import os
    
    # Locate the utils.py file in the trdg package
    utils_path = os.path.join(os.path.dirname(trdg.__file__), "utils.py")

    # Read and fix the file
    with open(utils_path, 'r') as file:
        content = file.read()

    if "image_font.getsize(text)[1]" in content:
        content = content.replace(
            "return image_font.getsize(text)[1]",
            "left, top, right, bottom = image_font.getbbox(text)\n    return bottom"
        )

        with open(utils_path, 'w') as file:
            file.write(content)
        print("Fixed utils.py successfully!")
    else:
        print("No need to fix utils.py")

In [4]:
fix_utils_file()

Fixed utils.py successfully!


# Creates synthetic word images with mixed distortion effects


In [ ]:
import os
import subprocess
import shutil
import random
import traceback
import tempfile
from PIL import Image
import numpy as np

GLOBAL_SEED = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# Folder paths
input_file = './sample_data/balanced_turkish_words.txt'
output_dir = './output/words/'
fonts_dir = './fonts/hw_fonts/'  # Directory containing example handwriting fonts, for example printed type fonts, change to './fonts/printed_fonts/


# Paths for texture(background) images
textures_dir = './textures/'
validated_textures_dir = './validated_textures/'
os.makedirs(textures_dir, exist_ok=True)
os.makedirs(validated_textures_dir, exist_ok=True)

# Create the output directory
os.makedirs(output_dir, exist_ok=True)

# total number of images to generate
total_target_images = 50
batch_size = 50

def prepare_texture_images():
    """
    Check textures directory, validate images, and create a new directory with
    properly processed images that avoid transparency issues.
    """
    os.makedirs(validated_textures_dir, exist_ok=True)

    # Check if original textures directory exists and has files
    if not os.path.exists(textures_dir):
        print(f"Textures directory {textures_dir} does not exist!")
        return validated_textures_dir, False

    texture_files = [f for f in os.listdir(textures_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not texture_files:
        print(f"No image files found in {textures_dir}")
        return validated_textures_dir, False

    print(f"Found {len(texture_files)} potential texture files. Processing...")
    valid_count = 0

    # Process each image file
    for img_file in texture_files:
        src_path = os.path.join(textures_dir, img_file)
        dst_path = os.path.join(validated_textures_dir, os.path.splitext(img_file)[0] + '.jpg')

        try:
            with Image.open(src_path) as img:
                # Handle palette images with transparency
                if img.mode == 'P' and 'transparency' in img.info:
                    img = img.convert('RGBA')

                # Convert to RGB mode to remove transparency
                if img.mode == 'RGBA':
                    background = Image.new('RGB', img.size, (255, 255, 255))
                    background.paste(img, mask=img.split()[3])
                    img = background
                elif img.mode != 'RGB':
                    img = img.convert('RGB')

                # Create a standard size canvas
                standard_size = (800, 600)
                new_img = Image.new('RGB', standard_size, (255, 255, 255))

                # Calculate position to center the original image
                position = ((standard_size[0] - img.width) // 2,
                           (standard_size[1] - img.height) // 2)

                # Paste original image onto the standard canvas
                new_img.paste(img, position)

                # Save as JPG (no transparency)
                new_img.save(dst_path, 'JPEG', quality=95)
                valid_count += 1

        except Exception as e:
            print(f"Error processing image {img_file}: {e}")

    print(f"Processed {valid_count} out of {len(texture_files)} texture images")

    if valid_count == 0:
        print("No valid textures found. Will use basic backgrounds instead.")

    return validated_textures_dir, valid_count > 0

def apply_post_processing(img_path):
    """Apply light Gaussian noise as post-processing to the generated images"""
    try:
        img = Image.open(img_path)
        if img.mode not in ['RGB', 'L']:
            img = img.convert('RGB')

        img_array = np.array(img, dtype=np.float32)

        # Add Gaussian noise
        mean = 0
        std = 10
        noise = np.random.normal(mean, std, img_array.shape)

        noisy_img_array = img_array + noise
        noisy_img_array = np.clip(noisy_img_array, 0, 255).astype(np.uint8)

        # Convert back to image
        noisy_img = Image.fromarray(noisy_img_array, mode=img.mode)
        noisy_img.save(img_path)
        return True

    except Exception as e:
        print(f"Error in post-processing (Gaussian noise) for image {img_path}: {e}")
        return False

def select_background_for_scenario(has_texture_backgrounds):
    """Choose a background type with weighted randomization"""
    if has_texture_backgrounds:
        bg_options = [
            ('0', 10),   # Noise - 10%
            ('3', 90)    # Image background - 90%
        ]
    else:
        bg_options = [
            ('0', 20),   # Noise - 20%
            ('1', 80)    # Plain white - 80%
        ]

    options, weights = zip(*bg_options)
    return random.choices(options, weights=weights, k=1)[0]

def select_varied_parameters():
    """Select varied parameters with weighted choices"""
    # Text colors with weights
    text_colors = [
        ('#000000', 40),  # Pure black - 40%
        ('#222222', 40),  # Dark gray - 40%
        ('#333333', 20)   # Medium dark gray - 20%
    ]

    # Margins (top,left,bottom,right)
    margins = [
        ('15,15,15,15', 33),
        ('10,20,10,20', 33),
        ('20,15,20,15', 34),
    ]

    # Space widths
    space_widths = [
        ('0.9', 30),   # Narrower - 30%
        ('1.0', 40),   # Normal - 40%
        ('1.1', 30)    # Wider - 30%
    ]

    # Character spacings
    char_spacings = [
        ('0', 60),  # No extra spacing - 60%
        ('1', 40)   # 1px spacing - 40%
    ]

    def weighted_choice(choices):
        options, weights = zip(*choices)
        return random.choices(options, weights=weights, k=1)[0]

    return {
        'text_color': weighted_choice(text_colors),
        'margins': weighted_choice(margins),
        'space_width': weighted_choice(space_widths),
        'character_spacing': weighted_choice(char_spacings)
    }

def generate_word_images(target_count, global_counter):
    """Main function to generate word images"""
    print(f"\nGenerating approximately {target_count} word images...")
    print(f"Using random seed: {GLOBAL_SEED}")

    # Prepare texture backgrounds if needed
    validated_textures_dir, has_texture_backgrounds = prepare_texture_images()
    print(f"Texture backgrounds available: {has_texture_backgrounds} in directory {validated_textures_dir}")

    # Track the total number of images generated
    total_generated = 0
    labels = {}

    # TRDG parameters
    blur = '2'
    skew_angle = '5'
    distorsion = '1'
    image_heights = [62, 64, 66]

    # Create batches
    remaining = target_count
    batch_number = 1

    while remaining > 0:
        current_batch_size = min(batch_size, remaining)
        print(f"Processing batch {batch_number} ({current_batch_size} images)...")

        with tempfile.TemporaryDirectory() as temp_dir:
            # Select background type for this batch
            background = select_background_for_scenario(has_texture_backgrounds)

            # Get varied parameters for this batch
            varied_params = select_varied_parameters()

            # Build the TRDG command
            cmd = [
                'trdg',
                '--dict', input_file,
                '--output_dir', temp_dir,
                '--count', str(current_batch_size),
                '--language', 'tr',
                '--extension', 'png',
                '--word_split',
                '--font_dir', fonts_dir,
                '--blur', blur,
                '--random_blur',
                '--distorsion', distorsion,
                '--distorsion_orientation', '1',
                '--skew_angle', skew_angle,
                '--random_skew',
                '--background', background,
                '--format', str(random.choice(image_heights)),
                '--name_format', '2',
                '--text_color', varied_params['text_color'],
                '--margins', varied_params['margins'],
                '--space_width', varied_params['space_width'],
                '--character_spacing', varied_params['character_spacing'],
            ]

            # Add alignment parameter
            alignment = random.choices(['0', '1', '2'], weights=[60, 20, 20])[0]
            cmd.extend(['--alignment', alignment])

            # If using image background, add image directory
            if background == '3' and has_texture_backgrounds:
                cmd.extend(['--image_dir', validated_textures_dir])

            # Run the command
            try:
                print(f"Executing command: {' '.join(cmd)}")
                result = subprocess.run(cmd, check=True, capture_output=True, text=True)
                print(f"Command executed successfully")
            except subprocess.CalledProcessError as e:
                print(f"Error executing command: {e}")
                print(f"Error output: {e.stderr}")
                continue

            # Check generated files
            generated_files = [f for f in os.listdir(temp_dir) if f.endswith('.png')]
            print(f"Generated {len(generated_files)} images in temporary directory")

            # Read the labels file
            labels_path = os.path.join(temp_dir, 'labels.txt')
            current_labels = {}

            if os.path.exists(labels_path):
                with open(labels_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        parts = line.strip().split(' ', 1)
                        if len(parts) == 2:
                            current_labels[parts[0]] = parts[1]

            # Copy, rename, and post-process files
            processed_count = 0
            for img_file, label in current_labels.items():
                # Create unique names
                scenario_img_num = total_generated + int(img_file.split('.')[0])
                global_img_num = global_counter + scenario_img_num

                scenario_name_png = f"{scenario_img_num}.png"

                src_path = os.path.join(temp_dir, img_file)
                dst_path = os.path.join(output_dir, scenario_name_png)

                try:
                    shutil.copy2(src_path, dst_path)
                    # Apply post-processing
                    apply_post_processing(dst_path)
                    labels[scenario_name_png] = label
                    processed_count += 1

                except Exception as e:
                    print(f"Error processing file {img_file}: {e}")

            # Update counts
            total_generated += processed_count
            remaining -= processed_count
            batch_number += 1

            print(f"Processed {processed_count} images in this batch")

    # Write the labels file
    labels_path = os.path.join(output_dir, 'labels.txt')
    with open(labels_path, 'w', encoding='utf-8') as f:
        for img_file, label in sorted(labels.items(), key=lambda x: int(x[0].split('.')[0])):
            f.write(f"{img_file} {label}\n")

    print(f"Total images generated: {total_generated}")
    print(f"Labels saved to {labels_path}")

    return total_generated, global_counter + total_generated

# Main execution
try:
    print(f"Using global random seed: {GLOBAL_SEED}")

    # Initialize global image counter
    global_image_counter = 0

    # Generate word images
    actual_count, global_image_counter = generate_word_images(
        total_target_images, global_image_counter
    )

    print(f"\nDataset generation complete. Total images: {actual_count}")
    print(f"Dataset available at: {output_dir}")

except Exception as e:
    print(f"Error generating dataset: {e}")
    print(traceback.format_exc())

Using global random seed: 42

Generating approximately 50 word images...
Using random seed: 42
No image files found in ./textures/
Texture backgrounds available: False in directory ./validated_textures/
Processing batch 1 (50 images)...
Executing command: trdg --dict ./sample_data/balanced_turkish_words.txt --output_dir C:\Users\LEGION\AppData\Local\Temp\tmpacfj4m2m --count 50 --language tr --extension png --word_split --font_dir ./fonts/hw_fonts/ --blur 2 --random_blur --distorsion 1 --distorsion_orientation 1 --skew_angle 5 --random_skew --background 1 --format 66 --name_format 2 --text_color #000000 --margins 15,15,15,15 --space_width 0.9 --character_spacing 1 --alignment 1
Command executed successfully
Generated 50 images in temporary directory


C:\Users\LEGION\AppData\Local\Temp\ipykernel_21984\1168418334.py:116: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  noisy_img = Image.fromarray(noisy_img_array, mode=img.mode)


Processed 50 images in this batch
Total images generated: 50
Labels saved to ./output/words/labels.txt

Dataset generation complete. Total images: 50
Dataset available at: ./output/words/
